[View in Colaboratory](https://colab.research.google.com/github/oleges1/practise/blob/master/add_learning/birds_imagenet.ipynb)

## Prepare gdrive

In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

from google.colab import auth
auth.authenticate_user()

from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

!mkdir -p drive
!google-drive-ocamlfuse drive

Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 18408 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8.1_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8.1) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .

In [2]:
!ls drive/birds/00_input/train/ -a

.  ..  gt.csv  images  images_reshaped


### Prepare directory for flow_from_directory method in keras

In [0]:
path_to_imgs = 'drive/birds/00_input/train/images'
path_to_csv =  'drive/birds/00_input/train/gt.csv'

In [5]:
import pandas as pd
import os
import numpy as np
import shutil

# source is the current directory
# Open dataset file
dataset = pd.read_csv(path_to_csv)
file_names = list(dataset['filename'].values)
img_labels = list(dataset['class_id'].values)

folders_to_be_created = np.unique(list(dataset['class_id'].values))

source = os.getcwd()

for new_path in folders_to_be_created:
    if not os.path.exists(path_to_imgs + '/' +  str(new_path)):
        os.makedirs(path_to_imgs + '/' +  str(new_path))

folders = folders_to_be_created.copy()

for f in range(len(file_names)):

  current_img = file_names[f]
  current_label = img_labels[f]

  shutil.move(path_to_imgs + '/' + str(current_img), path_to_imgs + '/' + str(current_label))

KeyboardInterrupt: ignored

In [0]:
!cp -r drive/birds/00_input/train/images drive/birds/00_input/train/images_validation

In [0]:
for new_path in folders_to_be_created:
    if not os.path.exists('drive/birds/00_input/train/validation' + '/' +  str(new_path)):
        os.makedirs('drive/birds/00_input/train/validation' + '/' +  str(new_path))

In [69]:
!ls drive/birds/00_input/train/images_validation/28 -l | wc

     49     437    2540


## Main  model

### Try diffrent model and test them using validation

In [0]:
path = 'drive/birds/00_input/train/images'

In [0]:
path_to_gt = 'drive/birds/00_input/train/gt.csv'

In [5]:
!pip3 install tqdm

In [6]:
import os
import numpy as np

from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing.image import ImageDataGenerator, load_img
from random import randint

from keras.models import Model, load_model
from keras.layers import Dense, GlobalAveragePooling2D, Activation, GlobalMaxPooling2D, Dropout, Conv2D, MaxPooling2D, Flatten, Reshape, BatchNormalization
from keras import backend as K

from keras.callbacks import LambdaCallback, ProgbarLogger

from keras.optimizers import Adam

from tqdm import tqdm_notebook, tqdm

#from run_tests import read_csv, save_csv

from skimage.transform import resize
from random import randint

Using TensorFlow backend.


In [0]:
def read_csv(filename):
    res = {}
    with open(filename) as fhandle:
        next(fhandle)
        for line in fhandle:
            filename, class_id = line.rstrip('\n').split(',')
            res[filename] = int(class_id)
    return res

def rotare(img, times):
    if times == 0:
      return img
    if times == 1:
      return np.rot90(img)
    if times == 2:
      return np.rot90(np.rot90(img))
    if times == 3:
      return np.rot90(np.rot90(np.rot90(img)))


def generator(train_img_dir, dataset, batch_size = 1, fast_run = True):
    x_batch = np.zeros((batch_size, 299, 299, 3), dtype = np.float32)
    y_batch = np.zeros((batch_size, 50))
    i = 0
    full_indx = np.arange(len(dataset))
    np.random.shuffle(full_indx)
    
    begin_batch = 0
    while True:
        if fast_run:
          filename_idxs = np.random.choice(len(dataset), batch_size)
        else:
          if begin_batch + batch_size < len(dataset):
              filename_idxs = full_indx[begin_batch : begin_batch + batch_size]
              begin_batch += batch_size
          else:
              filename_idxs = full_indx[begin_batch : len(dataset)]
              begin_batch = (begin_batch + batch_size) % len(dataset)
              filename_idxs += full_indx[0 : begin_batch]

        filenames = [list(dataset.keys())[filename_idx] for filename_idx in filename_idxs]
        for i in range(batch_size):
            image = np.array(load_img(train_img_dir + '/' +  filenames[i], target_size = (299, 299)), dtype = np.float32)
            if not fast_run:
                seed = randint(0, 3)
                image = rotare(image, seed)
            image -= np.mean(image, keepdims=True)
            image /= (np.std(image, keepdims=True) + 0.00001)
            x_batch[i] = image
            y_batch[i][dataset[filenames[i]]] = 1
        yield x_batch, y_batch

In [0]:
class data_storage():
  def __init__(self, path, path_to_gt, path_to_datas = ['drive/birds/x.npy', 'drive/birds/y.npy']):
      self.path = path
      self.path_to_gt = path_to_gt
      
      self.dataset = {}
      with open(path_to_gt) as fhandle:
          next(fhandle)
          for line in fhandle:
              filename, class_id = line.rstrip('\n').split(',')
              self.dataset[filename] = int(class_id)
      
      if path_to_datas != None:
          self.x = np.load(path_to_datas[0])
          self.y = np.load(path_to_datas[1])
          return

      self.x = np.zeros((len(self.dataset), 299, 299, 3), dtype = np.float32)
      self.y = np.zeros((len(self.dataset), 50))
      filenames = [list(self.dataset.keys())[filename_idx] for filename_idx in range(len(self.dataset))]
      for i in tqdm(range(len(self.dataset))):
          image = np.array(load_img(path + '/' +  filenames[i], target_size = (299, 299)), dtype = np.float32)
          #image -= np.mean(image, keepdims=True)
          #image /= (np.std(image, keepdims=True) + 0.00001)
          self.x[i] = image
          self.y[i][self.dataset[filenames[i]]] = 1
          
   
  # better to use keras flow
  def generator(self, fast_run = False):
      x_batch = np.zeros((batch_size, 299, 299, 3), dtype = np.float32)
      y_batch = np.zeros((batch_size, 50))
      full_indx = np.arange(len(dataset))
      np.random.shuffle(full_indx)
      begin_batch = 0
      while True:
        if fast_run:
            filename_idxs = np.random.choice(len(dataset), batch_size)
        else:
            if begin_batch + batch_size < len(dataset):
                filename_idxs = full_indx[begin_batch : begin_batch + batch_size]
                begin_batch += batch_size
            else:
                filename_idxs = full_indx[begin_batch : len(dataset)]
                begin_batch = (begin_batch + batch_size) % len(dataset)
                filename_idxs += full_indx[0 : begin_batch]
        filenames = [list(dataset.keys())[filename_idx] for filename_idx in filename_idxs]
        
        for i in range(batch_size):
            image = self.x[filename_idxs[i]]
            seed = randint(0, 3)
            image = rotare(image, seed)
            
            x_batch[i] = image
            y_batch[i][dataset[filenames[i]]] = 1
        yield x_batch, y_batch

In [0]:
full_data = data_storage(path, path_to_gt)

In [0]:
from sklearn.utils import shuffle
full_data.x, full_data.y = shuffle(full_data.x, full_data.y)

In [11]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:            12G        5.6G        151M        249M        7.0G        9.1G
Swap:            0B          0B          0B


In [0]:
# base_model = InceptionV3(weights='imagenet', include_top = False)

In [0]:
# base_model.predict(full_data.x[0].reshape([1, 299, 299, 3])).shape

In [0]:
def make_generators(batch_size = 32, validation = True):
    if validation:
      data_gen_args = dict(
                       #featurewise_center = True,
                       #featurewise_std_normalization = True,
                       samplewise_center = True,
                       samplewise_std_normalization = True,
                       horizontal_flip = True,
                       rotation_range=60.,
                       validation_split = 0.1)

      image_datagen = ImageDataGenerator(**data_gen_args)
      
      #image_datagen.fit(full_data.x)

      train_generator = image_datagen.flow(full_data.x, full_data.y, batch_size=batch_size, subset = 'training')

      validation_generator = image_datagen.flow(full_data.x, full_data.y, batch_size=250, subset = 'validation')

      return train_generator, validation_generator
    
    else:
      data_gen_args = dict(
                       #featurewise_center = True,
                       #featurewise_std_normalization = True,
                       samplewise_center = True,
                       samplewise_std_normalization = True,
                       horizontal_flip = True,
                       rotation_range=60.)

      image_datagen = ImageDataGenerator(**data_gen_args)
      
      #image_datagen.fit(full_data.x)

      train_generator = image_datagen.flow(full_data.x, full_data.y, batch_size=batch_size, subset = 'training')

      return train_generator

In [0]:
# model = None

# x = base_model.output
# part1 = Conv2D(2048, (2, 2), padding = 'same', activation = 'relu')(x)
# part1 = MaxPooling2D(pool_size=(2, 2))(part1)
# part1 = Dropout(0.15)(part1)

# part2 = Conv2D(4096, (2, 2), padding = 'same', activation = 'relu')(part1)
# part2 = MaxPooling2D(pool_size=(2, 2))(part2)
# part2 = Dropout(0.15)(part2)

# part3 = Conv2D(8192, (2, 2), padding = 'same', activation = 'relu')(part2)
# part3 = MaxPooling2D(pool_size=(2, 2))(part3)
# part3 = Dropout(0.15)(part3)

# pooled = GlobalAveragePooling2D()(part3)

# # let's add a fully-connected layer with dropout
# ending = Dense(1000, activation='relu')(pooled)
# ending = Dropout(0.3)(ending)
# # and a logistic layer -- let's say we have 200 classes
# predictions = Dense(50, activation='softmax')(ending)

# # this is the model we will train
# model = Model(inputs=base_model.input, outputs=predictions)

# # first: train only the top layers (which were randomly initialized)
# # i.e. freeze all convolutional InceptionV3 layers

# lr_main = 0.0001

# #for layer in base_model.layers:
# #    layer.trainable = False

# # compile the model (should be done *after* setting layers to non-trainable), also set small learning_rate
# model.compile(optimizer=Adam(lr = lr_main), loss='categorical_crossentropy', metrics=['accuracy'])
# #model.summary()

In [0]:
#!pip install -U --force-reinstall --no-dependencies git+https://github.com/datumbox/keras@bugfix/trainable_bn 

In [17]:
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top = False)
# add a global spatial average pooling layer
x = base_model.output
# x = Conv2D(2048, (4, 4), activation = 'relu')(x)
# x = BatchNormalization()(x)
# x = Conv2D(2048, (3, 3), activation = 'relu')(x)
# x = BatchNormalization()(x)
x = GlobalAveragePooling2D()(x)


# let's add a fully-connected layer
# x = Dropout(0.15)(x)
x = Dense(1024, activation='relu')(x)
#x = Dropout(0.4)(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(50, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
#for layer in base_model.layers:
#    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer=Adam(lr = 0.00005), loss='categorical_crossentropy', metrics = ['accuracy'])

87916544/87910968 [==============================] - 3s 0us/step


In [0]:
batch_sizes = [32, 36, 40, 48, 56, 60, 64, 68, 71]

In [19]:
for i, batch_size in enumerate(batch_sizes):
    train_generator, validation_generator = make_generators(batch_size = batch_size, validation=1)
    
    model.fit_generator(train_generator, steps_per_epoch= 2250 / batch_size, epochs=1, verbose = 1, validation_data = validation_generator, validation_steps = 1)
    if i > 2:
      model.save('drive/main_model' + str(i) + '.h5')

Epoch 1/1
71/70 [==============================] - 138s 2s/step - loss: 3.1498 - acc: 0.2910 - val_loss: 1.9827 - val_acc: 0.5000
Epoch 1/1
63/62 [==============================] - 108s 2s/step - loss: 1.4657 - acc: 0.6618 - val_loss: 1.0658 - val_acc: 0.6640
Epoch 1/1
57/56 [==============================] - 104s 2s/step - loss: 0.8077 - acc: 0.7881 - val_loss: 0.7599 - val_acc: 0.7920
Epoch 1/1
47/46 [==============================] - 107s 2s/step - loss: 0.5135 - acc: 0.8635 - val_loss: 0.6206 - val_acc: 0.8120
Epoch 1/1
41/40 [==============================] - 102s 2s/step - loss: 0.3682 - acc: 0.9070 - val_loss: 0.5879 - val_acc: 0.8280
Epoch 1/1
38/37 [==============================] - 104s 3s/step - loss: 0.2904 - acc: 0.9289 - val_loss: 0.5804 - val_acc: 0.8000
Epoch 1/1
36/35 [==============================] - 100s 3s/step - loss: 0.2312 - acc: 0.9452 - val_loss: 0.5446 - val_acc: 0.8440
Epoch 1/1
34/33 [==============================] - 104s 3s/step - loss: 0.2049 - acc: 0.96

KeyboardInterrupt: ignored

In [19]:
for i, batch_size in enumerate(batch_sizes[:-1]):
    train_generator = make_generators(batch_size = batch_size, validation=0)    
    model.fit_generator(train_generator, steps_per_epoch= 2500 / batch_size, epochs=1, verbose = 1)

Epoch 1/1
79/78 [==============================] - 143s 2s/step - loss: 2.9991 - acc: 0.3042
Epoch 1/1
70/69 [==============================] - 113s 2s/step - loss: 1.2575 - acc: 0.6999
Epoch 1/1
63/62 [==============================] - 111s 2s/step - loss: 0.6918 - acc: 0.8234
Epoch 1/1
53/52 [==============================] - 104s 2s/step - loss: 0.4945 - acc: 0.8571
Epoch 1/1
45/44 [==============================] - 105s 2s/step - loss: 0.3347 - acc: 0.9104
Epoch 1/1
42/41 [==============================] - 105s 2s/step - loss: 0.2519 - acc: 0.9486
Epoch 1/1
40/39 [==============================] - 103s 3s/step - loss: 0.2179 - acc: 0.9424
Epoch 1/1
37/36 [==============================] - 112s 3s/step - loss: 0.1725 - acc: 0.9595


In [0]:
model.save('drive/main_model.h5')

In [2]:
from keras.models import load_model
model = load_model('drive/temp_model.h5')

Using TensorFlow backend.


### No validation

In [0]:
#model = InceptionV3(weights='imagenet', classes=50, include_top = False)

base_model = InceptionV3(weights='imagenet', include_top = False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalMaxPooling2D()(x)

# let's add a fully-connected layer with dropout
x = Dense(1000, activation='relu')(x)
x = Dropout(0.25)(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(50, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable), also set small learning_rate
model.compile(optimizer=Adam(lr=0.00004), loss='categorical_crossentropy', metrics=['accuracy'])
#model.summary()

In [0]:
batch_size = 32
batches_per_training = 10

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [0]:
data_gen_args = dict(#rescale=1./256,
                     samplewise_center = True,
                     samplewise_std_normalization = True,
                     horizontal_flip = True,
                     rotation_range=90.)

def make_generator(batch_size = 32, data_gen_args = data_gen_args, path = path, batches_per_training = 10):
  
  image_datagen = ImageDataGenerator(**data_gen_args)
  train_generator = image_datagen.flow_from_directory(path, target_size=(299, 299), batch_size=batches_per_training*batch_size)
  
  return train_generator

In [0]:
batch_sizes = [32, 36, 40, 47, 54, 60, 66, 71]

In [0]:
batch_sizes = [69, 71]

In [17]:
for batch_size in batch_sizes:
    print('train on batch size:', batch_size)
    train_generator = make_generator(batch_size = batch_size)
    batches = 0
    for x_batch, y_batch in train_generator:
        model.fit(x_batch, y_batch,
          batch_size=batch_size, epochs=1, shuffle=False)
        batches += batches_per_training

        if (batches * batch_size) >= 2500:
            break

train on batch size: 32
Found 2500 images belonging to 50 classes.
Epoch 1/1
320/320 [==============================] - 33s 102ms/step - loss: 6.5379 - acc: 0.0219
Epoch 1/1
320/320 [==============================] - 13s 40ms/step - loss: 5.3813 - acc: 0.0500
Epoch 1/1
320/320 [==============================] - 13s 40ms/step - loss: 4.8115 - acc: 0.0625
Epoch 1/1
320/320 [==============================] - 13s 40ms/step - loss: 4.2825 - acc: 0.0938
Epoch 1/1
320/320 [==============================] - 13s 40ms/step - loss: 4.0895 - acc: 0.0687
Epoch 1/1
320/320 [==============================] - 13s 40ms/step - loss: 3.9113 - acc: 0.1125
Epoch 1/1
320/320 [==============================] - 13s 40ms/step - loss: 3.5866 - acc: 0.1562
Epoch 1/1
260/260 [==============================] - 13s 48ms/step - loss: 3.3307 - acc: 0.1923
train on batch size: 36
Found 2500 images belonging to 50 classes.
Epoch 1/1
360/360 [==============================] - 15s 41ms/step - loss: 2.8978 - acc: 0.2833
E

360/360 [==============================] - 15s 41ms/step - loss: 2.1406 - acc: 0.4167
Epoch 1/1
360/360 [==============================] - 15s 41ms/step - loss: 2.1323 - acc: 0.3917
Epoch 1/1
340/340 [==============================] - 17s 49ms/step - loss: 2.1076 - acc: 0.3882
train on batch size: 40
Found 2500 images belonging to 50 classes.
Epoch 1/1
400/400 [==============================] - 16s 40ms/step - loss: 1.5783 - acc: 0.5450
Epoch 1/1
400/400 [==============================] - 16s 41ms/step - loss: 1.6166 - acc: 0.5400
Epoch 1/1
400/400 [==============================] - 16s 40ms/step - loss: 1.5043 - acc: 0.5350
Epoch 1/1
400/400 [==============================] - 16s 41ms/step - loss: 1.6264 - acc: 0.5325
Epoch 1/1
400/400 [==============================] - 16s 40ms/step - loss: 1.5440 - acc: 0.5250
Epoch 1/1
400/400 [==============================] - 16s 40ms/step - loss: 1.4594 - acc: 0.5625
Epoch 1/1
100/100 [==============================] - 7s 65ms/step - loss: 1.460

470/470 [==============================] - 18s 38ms/step - loss: 1.1069 - acc: 0.6596
Epoch 1/1
470/470 [==============================] - 18s 38ms/step - loss: 1.1997 - acc: 0.6319
Epoch 1/1
150/150 [==============================] - 8s 53ms/step - loss: 1.3956 - acc: 0.6067
train on batch size: 54
Found 2500 images belonging to 50 classes.
Epoch 1/1
540/540 [==============================] - 28s 52ms/step - loss: 1.0342 - acc: 0.6611
Epoch 1/1
540/540 [==============================] - 20s 38ms/step - loss: 0.9664 - acc: 0.7148
Epoch 1/1
540/540 [==============================] - 20s 38ms/step - loss: 0.8556 - acc: 0.7426
Epoch 1/1
540/540 [==============================] - 20s 38ms/step - loss: 0.9245 - acc: 0.7037
Epoch 1/1
340/340 [==============================] - 13s 38ms/step - loss: 1.0979 - acc: 0.6676
train on batch size: 60
Found 2500 images belonging to 50 classes.
Epoch 1/1
600/600 [==============================] - 30s 50ms/step - loss: 0.8027 - acc: 0.7533
Epoch 1/1
600

660/660 [==============================] - 33s 50ms/step - loss: 0.7768 - acc: 0.7682
Epoch 1/1
660/660 [==============================] - 25s 37ms/step - loss: 0.6633 - acc: 0.7985
Epoch 1/1
660/660 [==============================] - 25s 37ms/step - loss: 0.6851 - acc: 0.7682
Epoch 1/1
520/520 [==============================] - 19s 37ms/step - loss: 0.6760 - acc: 0.7692
train on batch size: 71
Found 2500 images belonging to 50 classes.
Epoch 1/1
710/710 [==============================] - 35s 49ms/step - loss: 0.6347 - acc: 0.8085
Epoch 1/1
710/710 [==============================] - 26s 37ms/step - loss: 0.5575 - acc: 0.8127
Epoch 1/1
710/710 [==============================] - 26s 37ms/step - loss: 0.6569 - acc: 0.8014
Epoch 1/1
370/370 [==============================] - 16s 44ms/step - loss: 0.6053 - acc: 0.8216


In [0]:
model.save('drive/birds_model.hdf5')

In [0]:
model = load_model('drive/birds_model.hdf5')